# Extract data

In [1]:
from os.path import isfile, isdir
import zipfile

In [2]:
if not isfile('database.csv'):
    with zipfile.ZipFile('earthquake-database.zip', 'r') as the_zip:
        the_zip.extractall()
        the_zip.close()

# Explore data

Ref: https://www.kaggle.com/futurist/d/usgs/earthquake-database/mapping-of-earthquake-and-its-analysis

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# for installation
# conda install basemap
from mpl_toolkits.basemap import Basemap
import matplotlib.animation as animation
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

In [5]:
# load file
try:
    t_file = pd.read_csv('database.csv', encoding='ISO-8859-1')
    print('File load: Success')
except:
    print('File load: Failed')

File load: Success


In [6]:
# view data
t_file.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [7]:
t_file.shape

(23412, 21)

In [8]:
t_file.columns

Index(['Date', 'Time', 'Latitude', 'Longitude', 'Type', 'Depth', 'Depth Error',
       'Depth Seismic Stations', 'Magnitude', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status'],
      dtype='object')

In [20]:
t_file['Magnitude Type'].value_counts()

MW     7722
MWC    5669
MB     3761
MWB    2458
MWW    1983
MS     1702
ML       77
MWR      26
MD        6
MH        5
Name: Magnitude Type, dtype: int64

# Visualizing earthquackes (1965-2016) Animation

Ref: https://www.kaggle.com/artimous/d/usgs/earthquake-database/visualizing-earthquakes-via-animations

In [21]:
# plot
t_file['Year'] = t_file['Date'].str[6:]

In [22]:
fig = plt.figure(figsize = (10, 10))
fig.text(.8, .3, 'try animation', ha='right')
cmap = plt.get_cmap('coolwarm')

m = Basemap(projection='mill', llcrnrlat=-80, urcrnrlat=80, llcrnrlon=-180, urcrnrlon=180, lat_ts=20, resolution='c')
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='burlywood', lake_color='lightblue', zorder=1)
m.drawmapboundary(fill_color='lightblue')

START_YEAR = 1965
LAST_YEAR = 2016

points = t_file[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']][t_file['Year']==str(START_YEAR)]

x, y = m(list(points['Longitude']), list(points['Latitude']))
scat = m.scatter(x, y, s=points['Magnitude']*points['Depth']*0.3, marker='o', alpha=0.3, zorder=10, cmap=cmap)
year_text = plt.text(-170, 80, str(START_YEAR), fontsize=15)
plt.title("Earthquake visualisation (1965 - 2016)")
plt.close()

In [23]:
def update(frame_number):
    current_year = START_YEAR + (frame_number % (LAST_YEAR - START_YEAR + 1))
    year_text.set_text(str(current_year))
    points = t_file[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']][t_file['Year']==str(current_year)]
    x, y = m(list(points['Longitude']), list(points['Latitude']))
    color = points['Depth'] * points['Magnitude']
    scat.set_offsets(np.dstrack((x, y)))
    scat.set_sizes(points['Magnitude'] * points['Depth']*0.3)

In [24]:
# to install ffmpeg
# conda install -c menpo ffmpeg

In [25]:
ani = animation.FuncAnimation(fig, update, interval=1000, frames=LAST_YEAR - START_YEAR + 1)
ani.save('animation.gif', writer='imagemagick', fps=5)

ValueError: Cannot save animation: no writers are available. Please install ffmpeg to save animations.

In [26]:
import io
import base64

filename = 'animation.gif'

video = io.open(filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

FileNotFoundError: [Errno 2] No such file or directory: 'animation.gif'